In [7]:
import os
import re
import json
import shutil
import time
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain import LLMChain

# Charger les variables d'environnement
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("La clé API OpenAI n'a pas été trouvée dans le fichier .env")

feedback_dir = "data/feedback"
processed_dir = "data/feedback/feedback_processed"
jsonl_path = "data/decision_examples.jsonl"
os.makedirs(processed_dir, exist_ok=True)

def parse_feedback_file(filepath: str):
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()

    section_regex = r"-\s*Section actuelle\s*:\s*(\d+)"
    user_response_regex = r"-\s*Réponse utilisateur\s*:\s*(.*)"
    decision_regex = r"-\s*Décision prise\s*:\s*Section\s+(\d+)"
    section_content_regex = r"## Section\s*(.*?)\s*## Feedback"
    feedback_regex = r"## Feedback\s*(.*)"

    section_match = re.search(section_regex, content)
    user_response_match = re.search(user_response_regex, content)
    decision_match = re.search(decision_regex, content)
    section_content_match = re.search(section_content_regex, content, re.DOTALL)
    feedback_match = re.search(feedback_regex, content, re.DOTALL)

    if not (section_match and user_response_match and decision_match and section_content_match and feedback_match):
        print(f"Impossible de parser le feedback dans {filepath}")
        return None

    section = int(section_match.group(1))
    user_response = user_response_match.group(1).strip()
    taken_decision = int(decision_match.group(1))
    section_text = section_content_match.group(1).strip()
    feedback_text = feedback_match.group(1).strip()

    return {
        "filepath": filepath,
        "section": section,
        "user_response": user_response,
        "taken_decision": taken_decision,
        "section_text": section_text,
        "feedback_text": feedback_text
    }

def analyze_error_with_langchain(section: int, user_response: str, taken_decision: int, feedback_text: str, section_text: str) -> str:
    prompt_template = PromptTemplate(
        input_variables=["section", "user_response", "taken_decision", "feedback_text", "section_text"],
        template="""
Voici un retour utilisateur sur une décision prise par l'IA dans une histoire interactive.

- Section actuelle : {section}
- Réponse de l'utilisateur (choix souhaité) : {user_response}
- Décision prise par l'IA : {taken_decision}

Contenu de la section actuelle :
{section_text}

Feedback de l'utilisateur sur cette erreur :
{feedback_text}

Analyse de façon terre à terre et concise :
1. Indique quelle décision l'IA aurait dû prendre logiquement.
2. Dis si le feedback est justifié en fonction de la logique du jeu.
N'évoque aucune émotion, reste purement logique.
"""
    )

    llm = ChatOpenAI(
        temperature=0,
        openai_api_key=openai_api_key,
        model_name="gpt-4o-mini"
    )

    chain = LLMChain(llm=llm, prompt=prompt_template)
    analysis = chain.run(
        section=section,
        user_response=user_response,
        taken_decision=taken_decision,
        feedback_text=feedback_text,
        section_text=section_text
    )
    return analysis.strip()

def append_feedback_to_jsonl(section: int, user_response: str, taken_decision: int, analysis: str, feedback_text: str, section_text: str):
    system_content = "l'agent de décision doit comprendre les décisions de l'utilisateur en fonction du choix donné par le narrateur"
    user_content = (
        f"Section: {section}, l'utilisateur voulait '{user_response}', "
        f"l'IA a décidé section {taken_decision}. Section actuelle:\n{section_text}\n"
        f"Feedback: {feedback_text}"
    )
    assistant_content = analysis

    entry = {
        "messages": [
            {"role": "system", "content": system_content},
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": assistant_content}
        ]
    }

    print("Nouvelle entrée ajoutée au JSONL :")
    print(json.dumps(entry, ensure_ascii=False, indent=2))

    with open(jsonl_path, 'a', encoding='utf-8') as f:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

def process_all_feedback():
    files = [f for f in os.listdir(feedback_dir) if f.endswith(".md")]
    if not files:
        print("Aucun fichier de feedback à traiter.")
        return

    for filename in files:
        filepath = os.path.join(feedback_dir, filename)
        fb = parse_feedback_file(filepath)

        if fb is None:
            # On ne déplace pas le fichier s'il n'a pas été parsé
            print(f"Feedback non parsé pour {filename}, fichier laissé en place.")
            continue

        analysis = analyze_error_with_langchain(
            fb["section"],
            fb["user_response"],
            fb["taken_decision"],
            fb["feedback_text"],
            fb["section_text"]
        )

        append_feedback_to_jsonl(
            fb["section"],
            fb["user_response"],
            fb["taken_decision"],
            analysis,
            fb["feedback_text"],
            fb["section_text"]
        )

        shutil.move(filepath, os.path.join(processed_dir, filename))
        print(f"Feedback {filename} traité, ajouté au JSONL et fichier déplacé.")

# Exécuter le traitement de tous les feedbacks
process_all_feedback()


Aucun fichier de feedback à traiter.
